In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/hmeq.csv")

In [3]:
df["REASON"].fillna(value="DebtCon", inplace=True) # REASON : DebtCon = debt consolidation
df["JOB"].fillna(value="Other", inplace=True)      # JOB : Six occupational categories
df["DEROG"].fillna(value=0, inplace=True)          # DEROG : Number of major derogatory reports
df["DELINQ"].fillna(value=0, inplace=True)         # DELINQ : Number of delinquent credit lines
df.fillna(value=df.mean(), inplace=True)

In [4]:
df

,BAD,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,1,1100,25860.0000,39025.000000,HomeImp,Other,10.500000,0.0,0.0,94.366667,1.000000,9.000000,33.779915
1,1,1300,70053.0000,68400.000000,HomeImp,Other,7.000000,0.0,2.0,121.833333,0.000000,14.000000,33.779915
2,1,1500,13500.0000,16700.000000,HomeImp,Other,4.000000,0.0,0.0,149.466667,1.000000,10.000000,33.779915
3,1,1500,73760.8172,101776.048741,DebtCon,Other,8.922268,0.0,0.0,179.766275,1.186055,21.296096,33.779915
4,0,1700,97800.0000,112000.000000,HomeImp,Office,3.000000,0.0,0.0,93.333333,0.000000,14.000000,33.779915
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5955,0,88900,57264.0000,90185.000000,DebtCon,Other,16.000000,0.0,0.0,221.808718,0.000000,16.000000,36.112347
5956,0,89000,54576.0000,92937.000000,DebtCon,Other,16.000000,0.0,0.0,208.692070,0.000000,15.000000,35.859971
5957,0,89200,54045.0000,92924.000000,DebtCon,Other,15.000000,0.0,0.0,212.279697,0.000000,15.000000,35.556590
5958,0,89800,50370.0000,91861.000000,DebtCon,Other,14.000000,0.0,0.0,213.892709,0.000000,16.000000,34.340882


Применяем WoE+LogReg

In [5]:
# !pip install woe

In [6]:
import os
import numpy as np
import woe.feature_process as fp
import woe.GridSearch as gs

In [7]:
config_path = os.getcwd()+'\\../data/config_woe.csv'
data_path = os.getcwd()+'\\../data/woe_data.csv'
feature_detail_path = os.getcwd()+'\\features_detail.csv'
rst_pkl_path = os.getcwd()+'\\woe_rule.pkl'
# train woe rule
feature_detail,rst = fp.process_train_woe(infile_path=data_path
                                           ,outfile_path=feature_detail_path
                                           ,rst_path=rst_pkl_path
                                           ,config_path=config_path)

# proc woe transformation
woe_train_path = os.getcwd()+'\\dataset_train_woed.csv'
fp.process_woe_trans(data_path,rst_pkl_path,woe_train_path,config_path)
# here i take the same dataset as test dataset
woe_test_path = os.getcwd()+'\\dataset_test_woed.csv'
fp.process_woe_trans(data_path,rst_pkl_path,woe_test_path,config_path)

print('###TRAIN SCORECARD MODEL###')
params = {}
params['dataset_path'] = woe_train_path
params['validation_path'] = woe_test_path
params['config_path'] = config_path

params['df_coef_path'] = os.getcwd()+'\\df_model_coef_path.csv'
params['pic_coefpath'] = os.getcwd()+'\\model_coefpath.png'
params['pic_performance'] = os.getcwd()+'\\model_performance_path.png'
params['pic_coefpath_title'] = 'model_coefpath'
params['pic_performance_title'] = 'model_performance_path'

params['var_list_specfied'] = []
params['cs'] = np.logspace(-4, -1,40)
for key,value in params.items():
   print(key,': ',value)
gs.grid_search_lr_c_main(params)

run into process_train_woe: 
 Thu Feb 17 18:32:00 2022
------------------Changing Feature Dtypes-------------------
target                                         int64
LOAN                                           int64
MORTDUE                                        int64
VALUE                                          int64
REASON                                        object
JOB                                           object
YOJ                                            int64
DEROG                                          int64
DELINQ                                         int64
CLAGE                                          int64
NINQ                                           int64
CLNO                                           int64
DEBTINC                                        int64
------------Variable Dtypes Have Been Specified-------------
process woe transformation of continuous variables: 
 Thu Feb 17 18:32:00 2022
cfg.global_bt 1189
cfg.global_gt 4771
--------------pro

C:\Users\admin\anaconda3\envs\snakes\lib\site-packages\woe\feature_process.py:414: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[var] = cpvar
C:\Users\admin\anaconda3\envs\snakes\lib\site-packages\woe\feature_process.py:414: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[var] = cpvar


------------------Changing Feature Dtypes-------------------
target                                         int64
LOAN                                           int64
MORTDUE                                        int64
VALUE                                          int64
REASON                                        object
JOB                                           object
YOJ                                            int64
DEROG                                          int64
DELINQ                                         int64
CLAGE                                          int64
NINQ                                           int64
CLNO                                           int64
DEBTINC                                        int64
------------Variable Dtypes Have Been Specified-------------
###TRAIN SCORECARD MODEL###
dataset_path :  C:\Users\admin\PycharmProjects\CREDIT_SCORING\notebooks\dataset_train_woed.csv
validation_path :  C:\Users\admin\PycharmProjects\CREDIT_SCORING\n

(0.0028942661247167516, 0.6401432893115276)

In [9]:
feature_detail

,var_name,split_list,sub_total_sample_num,positive_sample_num,negative_sample_num,sub_total_num_percentage,positive_rate_in_sub_total,woe_list,iv_list,iv
0,LOAN,"(-INF,5900.0]",304,142,162,0.051007,0.467105,1.257674,0.107497,0.249598
1,LOAN,"(5900.0,7600.0]",299,87,212,0.050168,0.290970,0.498766,0.014332,0.249598
2,LOAN,"(7600.0,10000.0]",608,137,471,0.102013,0.225329,0.154567,0.002551,0.249598
3,LOAN,"(10000.0,11400.0]",365,57,308,0.061242,0.156164,-0.297604,0.004945,0.249598
4,LOAN,"(11400.0,12600.0]",347,76,271,0.058221,0.219020,0.118059,0.000840,0.249598
...,...,...,...,...,...,...,...,...,...,...
0,JOB,"[Office, missing]",1227,148,1079,0.205872,0.120619,-0.597134,0.060719,0.115381
1,JOB,[ProfExe],1276,212,1064,0.214094,0.166144,-0.223761,0.010005,0.115381
2,JOB,[Other],2388,554,1834,0.400671,0.231993,0.192353,0.015683,0.115381
3,JOB,[Mgr],767,179,588,0.128691,0.233377,0.200103,0.005463,0.115381


In [11]:
train_woed = pd.read_csv(woe_train_path)
train_woed

,Unnamed: 0,target,LOAN,MORTDUE,VALUE,REASON,JOB,YOJ,DEROG,DELINQ,CLAGE,NINQ,CLNO,DEBTINC
0,0,1,1.257674,0.505241,6.314432e-08,0.115037,0.192353,-0.088067,-0.220790,-0.429947,0.265123,-0.062557,0.084531,1.880533
1,1,1,1.257674,0.058210,6.314432e-08,0.115037,0.192353,-0.088067,-0.220790,1.672861,0.167122,-0.295364,0.084531,1.880533
2,2,1,1.257674,0.329934,6.314432e-08,0.115037,0.192353,0.146251,-0.220790,-0.429947,0.376445,-0.062557,0.084531,1.880533
3,3,1,1.257674,0.031860,6.314432e-08,0.115037,-0.597134,-0.545416,-0.575979,-0.564371,0.308074,-0.368414,0.084531,1.880533
4,4,0,1.257674,-0.540205,6.314432e-08,0.115037,-0.597134,0.358184,-0.220790,-0.429947,0.265123,-0.295364,0.084531,1.880533
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5955,5955,0,0.216998,-0.309722,6.314432e-08,-0.062752,0.192353,-0.088067,-0.220790,-0.429947,-0.219994,-0.295364,0.084531,-1.345087
5956,5956,0,0.216998,0.159754,6.314432e-08,-0.062752,0.192353,-0.088067,-0.220790,-0.429947,-0.412265,-0.295364,0.084531,-1.345087
5957,5957,0,0.216998,0.159754,6.314432e-08,-0.062752,0.192353,-0.088067,-0.220790,-0.429947,-0.412265,-0.295364,0.084531,-1.345087
5958,5958,0,0.216998,0.159754,6.314432e-08,-0.062752,0.192353,-0.088067,-0.220790,-0.429947,-0.412265,-0.295364,0.084531,-1.345087


In [17]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X = train_woed.drop(columns=["target", 'Unnamed: 0'])
print(X.columns)
y = train_woed['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=1488)
log_reg = LogisticRegression(C=0.6401432893115276)
log_reg.fit(X_train, y_train)

Index(['LOAN', 'MORTDUE', 'VALUE', 'REASON', 'JOB', 'YOJ', 'DEROG', 'DELINQ',
       'CLAGE', 'NINQ', 'CLNO', 'DEBTINC'],
      dtype='object')


LogisticRegression(C=0.6401432893115276)

In [18]:
log_reg.intercept_

array([-1.38939708])

In [19]:
log_reg.coef_

array([[ 5.84048141e-01,  9.74568964e-01, -1.94506592e-08,
         3.35724350e-01,  8.05381380e-01,  8.31705425e-01,
         7.28168171e-01,  9.14468783e-01,  1.11166149e+00,
         3.41952263e-01,  5.17514171e-01,  8.83795362e-01]])

In [20]:
y_pred = log_reg.predict(X_test)
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

print("Accuracy score : ", accuracy_score(y_test, y_pred))
print("F1 score : ", f1_score(y_test, y_pred, average="macro"))
print("Precision score : ", precision_score(y_test, y_pred, average="macro"))
print("Recall score : ", recall_score(y_test, y_pred, average="macro"))

Accuracy score :  0.8647686832740213
F1 score :  0.7799968716466912
Precision score :  0.8123993063325129
Recall score :  0.7582954061904498
